<a href="https://colab.research.google.com/github/krjohnn/LU-VTL-25m/blob/main/notebooks/MSP/seq2seq_punctuation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comma Restoration with Encoder-Decoder Models (Seq2Seq)
This notebook fine-tunes sequence-to-sequence transformers to restore commas by generating the punctuated sentence from an unpunctuated input. During training, commas are removed from the source text; the target is the original, correctly punctuated sentence. At inference, the model takes clean, comma-stripped text and outputs the version with commas inserted end-to-end—no token-level labels needed.

Models to try:
- https://huggingface.co/google/mt5-small
- https://huggingface.co/google/byt5-small

# Prepare environment

In [ ]:
# Authenticate with Weights & Biases to enable logging and experiment tracking.
# Comment out the following lines if you don't want to use W&B.
!pip install wandb
import wandb
import os
from google.colab import userdata
# Get your 86-character key from secrets
api_key = userdata.get('WANDB_API_KEY')

# Set the environment variable manually as a backup
os.environ["WANDB_API_KEY"] = api_key

# Login (relogin=True helps clear out any old 40-character errors)
wandb.login(key=api_key, relogin=True)

In [ ]:
# Check if a CUDA device is available
!pip install torch
import torch
if torch.cuda.is_available():
    print('CUDA device:', torch.cuda.get_device_name(0), torch.cuda.get_device_capability(0), 'bf16', torch.cuda.is_bf16_supported(False))
    free_mem, total_mem = torch.cuda.mem_get_info(torch.device('cuda:0'))
    print(f'Memory: {free_mem / 1024 ** 2:.2f} MB free / {total_mem / 1024 ** 2:.2f} MB total')
else:
    print('No CUDA device available')

In [ ]:
!python -V
!pip -V
!pip install numpy transformers[torch] scikit-learn datasets wandb

In [ ]:
import difflib
import re

import numpy as np
import requests
import torch
from datasets import load_dataset
from contextlib import nullcontext
from torch.utils.data import DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    set_seed, PreTrainedTokenizerBase, Seq2SeqTrainer,
)
import wandb

# Prepare dataset
Raw sentences from the Latvian Universal Dependencies (LVTB) corpus: https://universaldependencies.org/treebanks/lv_lvtb/index.html

In [ ]:
def fetch_ud_texts(split, seed=42):
    conllu = requests.get(f'https://raw.githubusercontent.com/UniversalDependencies/UD_Latvian-LVTB/r2.16/lv_lvtb-ud-{split}.conllu').text
    texts = [line[9:].strip() for line in conllu.splitlines() if line.startswith('# text = ')]
    if seed:
        import random
        random.Random(seed).shuffle(texts)
    return texts

def prepare_data(max_chars=200, dev_txt='dev.txt', train_txt='train.txt'):
    # Download UD Latvian splits, filter by mBERT token count, and save plain .txt files.
    dev_texts = fetch_ud_texts('dev')
    train_texts = fetch_ud_texts('train')

    if max_chars:
        # Filter out long sentences to avoid truncation
        print('Sentence lengths before filtering:', 'DEV', len(dev_texts), 'TRAIN', len(train_texts))
        dev_texts = [t for t in dev_texts if len(t) <= max_chars]
        train_texts = [t for t in train_texts if len(t) <= max_chars]
    print('Dataset sentence lengths:', 'DEV', len(dev_texts), 'TRAIN', len(train_texts))

    with open(dev_txt, 'w') as f:
        for t in dev_texts: f.write(t + '\n')
    with open(train_txt, 'w') as f:
        for t in train_texts: f.write(t + '\n')

    return dev_texts, train_texts

dev_texts, train_texts = prepare_data()
print(*train_texts[:5], sep='\n')

# Utilities

In [ ]:
# def remove_commas(s) -> str:
#     return re.sub(r'\s*,+\s*', ' ', s)

def remove_commas(s) -> str:
    # Logic: Replace commas with a space, BUT NOT if they are between digits (3,2)
    # The regex checks: Is there a digit before AND after? If so, keep it.
    return re.sub(r'(?<!\d)\s*,+\s*|\s*,+\s*(?!\d)', ' ', s)

# Tokenization

In [ ]:
def test_tokenization(model=None):
    s = 'Vēl 9% sacīja, ka nav izlēmuši kā balsot, bet 3,2% atteicās atbildēt.'
    if model:
        print('Tokenizer', model)
        t = AutoTokenizer.from_pretrained(model)
        print('Encoded sample:', t(s))
        print('Encoded sample - subword units:', t.convert_ids_to_tokens(t.encode(s)))
        lengths = sorted([len(t.encode(seq)) for seq in train_texts])
        print(f'Max {max(lengths)}, min {min(lengths)}, avg {sum(lengths)/len(lengths)}')
        print(f'95% length: {lengths[int(len(lengths) * 0.95)]}')
        print(f'99% length: {lengths[int(len(lengths) * 0.99)]}')
        print(f'99.9% length: {lengths[int(len(lengths) * 0.999)]}')

test_tokenization('google/mt5-small')
test_tokenization('google/byt5-small')

Tokenize and format dataset for model training and evaluation

In [ ]:
def build_dataset(*, tokenizer, train_file='train.txt', dev_file='dev.txt', train_samples=None, dev_samples=None, max_length=100):
    ds = load_dataset('text', data_files={'train': train_file, 'dev': dev_file})
    if train_samples:
        ds['train'] = ds['train'].take(train_samples)
    if dev_samples:
        ds['dev'] = ds['dev'].take(dev_samples)

    def _encode_examples(batch):
        targets = batch['text']
        sources = [remove_commas(t) for t in targets]
        enc_in = tokenizer(sources, max_length=max_length, truncation=True)
        enc_out = tokenizer(text_target=targets, max_length=max_length, truncation=True)
        enc_in['labels'] = enc_out['input_ids']
        return enc_in

    ds_encoded = ds.map(_encode_examples, batched=True, remove_columns=ds['train'].column_names)

    return ds_encoded

tok = AutoTokenizer.from_pretrained('google/mt5-small')
ds = build_dataset(tokenizer=tok, train_samples=10, dev_samples=10)
collator = DataCollatorForSeq2Seq(tok)
loader = DataLoader(ds['train'], batch_size=3, shuffle=False, collate_fn=collator)
batch = next(iter(loader))
print(batch)

# Metrics for token classification.
*F1-score* (the harmonic mean of precision and recall) specifically for the COMMA class gives a more honest view of model quality:
  - Precision: when the model predicts COMMA, is it right?
  - Recall: does the model catch most of the true commas?
  - F1: balances both, penalizing if one is much lower.

*Changes* - the percentage of sentences where the model introduced modifications that were **not desired**.  
This highlights over-correction: even if the model achieves good precision/recall on commas, a high *Changes* value means it is altering sentences unnecessarily, reducing usability in practice.

In [ ]:
def align_and_count_commas(hyp: str, ref: str) -> tuple[int, int, int]:
    sm = difflib.SequenceMatcher(a=hyp, b=ref, autojunk=False)
    tp = fp = fn = 0
    for tag, i1, i2, j1, j2 in sm.get_opcodes():
        if tag == 'equal':
            tp += hyp[i1:i2].count(',')
        else:
            fp += hyp[i1:i2].count(',')
            fn += ref[j1:j2].count(',')
    return tp, fp, fn


def eval_commas(refs: list[str], preds: list[str], verbose=False) -> dict[str, float]:
    verbose_changes_limit = 5
    tp = fp = fn = changes = exact = 0
    for hyp, ref in zip(preds, refs):
        tpp, fpp, fnn = align_and_count_commas(hyp, ref)
        tp += tpp; fp += fpp; fn += fnn
        if hyp == ref:
            exact += 1
        is_changed = re.sub(r'[\s,]', '', hyp) != re.sub(r'[\s,]', '', ref)
        if is_changed:
            changes += 1

        if verbose and verbose_changes_limit > 0 and is_changed:
            print('--- Changed')
            print('REF:', ref)
            print('OUT:', hyp)
            verbose_changes_limit -= 1

    p = tp / (tp + fp) if (tp + fp) else 0.0
    r = tp / (tp + fn) if (tp + fn) else 0.0
    f1 = 2 * p * r / (p + r) if (p + r) else 0.0
    return {
        'f1': f1, 'p': p, 'r': r,
        'changes': (changes / len(preds) if preds else 0.0),
        'exact':  (exact / len(preds) if preds else 0.0),
        'tp': tp, 'fp': fp, 'fn': fn,
    }


def compute_metrics(eval_preds, tokenizer, verbose=False):
    preds, labels = eval_preds
    pad_id = tokenizer.pad_token_id

    # Replace ignore index in preds
    labels = np.where(labels != -100, labels, pad_id)
    preds = np.where(preds != -100, preds, pad_id)

    decoded_preds  = tokenizer.batch_decode(preds,  skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    return eval_commas(decoded_labels, decoded_preds, verbose=verbose)

print(eval_commas(
    ['Labi atpūšamies, draugi mīļie, un lai veiksmīga, sportiska, panākumiem bagāta mums visiem jaunā vasaras sezona!'],
    ['Labi atpūšamies, draugi, mīļie un lai veiksmīga, sportiska, panākumiem bagāta mums visiem jaunā vasaras sezona!'],
))
print(eval_commas(
    ['Labi atpūšamies, draugi mīļie, un lai veiksmīga, sportiska, panākumiem bagāta mums visiem jaunā vasaras sezona!'],
    ['Labi atpūšamies draugi mīļie un lai veiksmīgas, sportiska, panākumiem bagāta mums visiem jaunā vasaras sezona.'],
))

# Inference
Given plain text, we strip commas, tokenize with word boundaries, run the model, and insert commas after tokens labeled COMMA.

In [ ]:
def process_text(text, model, tokenizer: PreTrainedTokenizerBase, max_len=120, verbose=True):
    model.eval()
    device = next(model.parameters()).device
    source = remove_commas(text)
    inputs = tokenizer([source], return_tensors='pt', truncation=True, max_length=max_len).to(device)
    with torch.no_grad():
        gen = model.generate(
            **inputs,
            max_new_tokens=max_len,
        )
    result = tokenizer.decode(gen[0], skip_special_tokens=True)
    if verbose:
        print(f'REF: {text}')
        print(f' IN: {source}')
        print(f'OUT: {result}')
    return result

# Model fine-tuning
- Track loss curves, gradient norms, and evaluation metrics over time
- Use an appropriate optimizer and learning rate schedule (e.g., warmup + decay)
- Watch for overfitting (gap between train and eval performance)
- Adjust batch size, accumulation steps, or precision (fp16/bf16) if needed
- Save best checkpoints based on validation metric (e.g., F1)

In [ ]:
def main(
    name='punctuator',
    base_model='google/mt5-small',
    max_len=80,
    seed=42,
    verbose=True,
    lr=1e-3,
    bs=8,
    train_samples=None,
    dev_samples=100,
    epochs=3,
    report_wandb=True,
    wandb_group=None
):
    if report_wandb and not wandb.api.api_key:
        print('Not authenticated with W&B')
        report_wandb = False

    with wandb.init(project='punctuator', group=wandb_group, name=name) if report_wandb else nullcontext():
        print('Train:', locals())
        set_seed(seed)
        tokenizer = AutoTokenizer.from_pretrained(base_model)

        # Load dataset
        ds = build_dataset(tokenizer=tokenizer, train_samples=train_samples, dev_samples=dev_samples, max_length=max_len)

        # Initialize base model for tokenize sequence to sequence task
        model = AutoModelForSeq2SeqLM.from_pretrained(base_model)
        model.config.use_cache = False

        # Define training hyperparameters
        training_args = Seq2SeqTrainingArguments(
            output_dir=name,
            report_to='wandb' if report_wandb else 'none',
            learning_rate=lr,
            per_device_train_batch_size=bs,
            per_device_eval_batch_size=bs,
            num_train_epochs=epochs,
            warmup_ratio=0.05,
            gradient_accumulation_steps=1,
            gradient_checkpointing=True,
            bf16=True,
            logging_steps=50,
            save_total_limit=1,
            save_strategy='epoch',
            eval_strategy='epoch',
            eval_accumulation_steps=1,
            load_best_model_at_end=True,
            metric_for_best_model='f1',
            greater_is_better=True,
            predict_with_generate=True,
            generation_max_length=max_len * 2,
        )

        trainer = Seq2SeqTrainer(
            model=model,
            args=training_args,
            train_dataset=ds['train'],
            eval_dataset=ds['dev'],
            processing_class=tokenizer,
            data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
            compute_metrics=lambda p: compute_metrics(p, tokenizer, verbose=verbose),
        )

        # Actual training
        trainer.train()
        trainer.save_model(name)
        tokenizer.save_pretrained(name)

        process_text('Vēl 9% sacīja, ka nav izlēmuši kā balsot, bet 3,2% atteicās atbildēt.', trainer.model, tokenizer, max_len=max_len)

main('mt5_punctuator_sample', train_samples=1000)

# Inference

In [ ]:
m = AutoModelForSeq2SeqLM.from_pretrained('mt5_punctuator_sample')
t = AutoTokenizer.from_pretrained('mt5_punctuator_sample')
process_text('Nogalināt nedrīkst, apžēlot!', m, t)
process_text('Palielināt izdevumus nedrīkst taupīt!', m, t)